In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-8b-instant"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="my_collection")

In [4]:
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids = ['id1', 'id2']
)

C:\Users\Vivek\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [04:55<00:00, 282kiB/s] 


In [5]:
all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This document is about New York',
  'This document is about Delhi'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None, None]}

In [6]:

documents = collection.get(ids=["id1"])
documents

{'ids': ['id1'],
 'embeddings': None,
 'documents': ['This document is about New York'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None]}

In [7]:
results = collection.query(
    query_texts=['Query is about big apple'],
    n_results=2
)
results

{'ids': [['id1', 'id2']],
 'embeddings': None,
 'documents': [['This document is about New York',
   'This document is about Delhi']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.6312181949615479, 1.867558479309082]]}

In [8]:

collection.delete(ids=all_docs['ids'])
collection.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

In [9]:

collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids=["id3", "id4"],
    metadatas=[
        {"url": "https://en.wikipedia.org/wiki/New_York_City"},
        {"url": "https://en.wikipedia.org/wiki/New_Delhi"}
    ]
)

In [10]:
results = collection.query(
    query_texts=["Query is about Chhole Bhature"],
    n_results=2
)
results

{'ids': [['id4', 'id3']],
 'embeddings': None,
 'documents': [['This document is about Delhi',
   'This document is about New York']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'url': 'https://en.wikipedia.org/wiki/New_Delhi'},
   {'url': 'https://en.wikipedia.org/wiki/New_York_City'}]],
 'distances': [[1.5588479042053223, 1.8114913702011108]]}

In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/lead-software-engineer/job/R-62911")
page_data = loader.load().pop().page_content
print(page_data)






















Lead Software Engineer










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Sel

In [17]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [18]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer',
 'experience': '8+ years of experience in software development, preferably in a scaled agile environment',
 'skills': ['Extensive experience in the Software Development Life Cycle, including system cloud architectural design',
  'Proficient in front-end development using Angular/React with TypeScript/JavaScript, as well as backend development with .NET Core (EF Core)',
  'Skilled in database design and management with SQL/NoSQL databases such as SQL Server and Cosmos DB',
  'Adept at developing REST APIs and Serverless Functions using Azure Functions or AWS Lambda',
  'Familiar with CI/CD pipelines and data engineering using Databricks'],
 'description': 'The Lead Software Engineer will be responsible for designing, developing, and maintaining software applications and data infrastructure. This role requires a blend of software engineering and data engineering skills to ensure the delivery of high-quality solutions that meet or solve business needs or

In [19]:
type(json_res)

dict

In [20]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [21]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [22]:
job = json_res
job['skills']

['Extensive experience in the Software Development Life Cycle, including system cloud architectural design',
 'Proficient in front-end development using Angular/React with TypeScript/JavaScript, as well as backend development with .NET Core (EF Core)',
 'Skilled in database design and management with SQL/NoSQL databases such as SQL Server and Cosmos DB',
 'Adept at developing REST APIs and Serverless Functions using Azure Functions or AWS Lambda',
 'Familiar with CI/CD pipelines and data engineering using Databricks']

In [23]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}]]

In [26]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        Your job is to write a cold email to the client regarding the job mentioned above describing the my capability 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase my portfolio: {link_list}
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Lead Software Engineer with Expertise in Scaled Agile Environment

Dear Hiring Manager,

I came across the job description for a Lead Software Engineer and was impressed by the requirements. With 8+ years of experience in software development, I believe I can fulfill your needs and contribute to the success of your team.

As a seasoned software engineer, I possess extensive experience in the Software Development Life Cycle, including system cloud architectural design. My proficiency in front-end development using Angular/React with TypeScript/JavaScript, as well as backend development with .NET Core (EF Core), aligns with your requirements. I am also skilled in database design and management with SQL/NoSQL databases such as SQL Server and Cosmos DB.

I have a proven track record of developing REST APIs and Serverless Functions using Azure Functions or AWS Lambda, and I am familiar with CI/CD pipelines and data engineering using Databricks. My expertise in designing, developing

In [1]:
print("Subject: Lead Software Engineer Application for High-Quality Solutions\n\nDear Hiring Manager,\n\nI came across the Lead Software Engineer role at your organization, and I'm excited to apply for the position. With my extensive experience in software development and data engineering, I'm confident that I can deliver high-quality solutions that meet or solve business needs.\n\nAs a full-stack developer with a strong background in blockchain, machine learning, and software development, I've developed a range of applications, including job searching platforms, NFT marketplaces, and decentralized autonomous organizations. My expertise in programming languages such as C, C++, Python, Javascript, and Go, along with my experience in cloud services and CICD pipelines, make me a strong fit for this role.\n\nI'm particularly drawn to this position because it requires a blend of software engineering and data engineering skills, which aligns with my skills and experience. My proficiency in data management tools such as Python, SQL, and PySpark, as well as my experience with databases like Oracle, MySQL, and MongoDB, will enable me to design, develop, and maintain software applications and data infrastructure effectively.\n\nI'm impressed by the organization's commitment to delivering high-quality solutions, and I'm excited about the opportunity to contribute my skills and experience to the team. My strong problem-solving abilities, attention to detail, and excellent communication and collaboration skills will enable me to work effectively with business stakeholders to deliver solutions that meet their needs.\n\nI've attached my resume for your review. I'd welcome the opportunity to discuss my application and how I can contribute to the organization's success. Please feel free to contact me at your convenience.\n\nThank you for considering my application.\n\nBest regards,\n[Your Name]")

Subject: Lead Software Engineer Application for High-Quality Solutions

Dear Hiring Manager,

I came across the Lead Software Engineer role at your organization, and I'm excited to apply for the position. With my extensive experience in software development and data engineering, I'm confident that I can deliver high-quality solutions that meet or solve business needs.

As a full-stack developer with a strong background in blockchain, machine learning, and software development, I've developed a range of applications, including job searching platforms, NFT marketplaces, and decentralized autonomous organizations. My expertise in programming languages such as C, C++, Python, Javascript, and Go, along with my experience in cloud services and CICD pipelines, make me a strong fit for this role.

I'm particularly drawn to this position because it requires a blend of software engineering and data engineering skills, which aligns with my skills and experience. My proficiency in data management 